In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import numpy as np
import pandas as pd
import datetime as dt
import random
from collections import Counter

In [ ]:
dir = 'drive/My Drive/Colab Notebooks/CSC/Projects/REZ_8/'
with open(dir + 'full_db(18-05-21).json') as json_file:
    flats = json.load(json_file)

In [ ]:
def prep_digit(number):
    '''preprocess number'''
    digit = ''
    thousands = False
    for symb in number:
        if symb.isdigit():
            digit += symb
    if number.find('тыс') != -1:
        digit += '000'
    return int(digit)    

In [ ]:
for id in random.sample(flats.keys(), 10):
    print(id, flats[id]['address'].split(', '))
    print('-------')

3681626 ['Санкт-Петербург', 'Выборгский район', 'Просвещения пр-кт']
-------
3695052 ['Санкт-Петербург', 'Кировский район', 'Ленинский проспект', '75 к 1']
-------
3625901 ['Санкт-Петербург', 'Выборгский район', 'Просвещения пр-кт']
-------
3665671 ['Ленинградская область', 'Выборгский район', 'г. Выборг', 'ул. Спортивная', '4']
-------
3626264 ['Санкт-Петербург', 'Выборгский район', 'Просвещения пр-кт']
-------
3627393 ['Ленинградская область', 'Всеволожский район', 'Бугры', 'Дорога в Мурино']
-------
3663730 ['Санкт-Петербург', 'Приморский район', 'Глухарская ул.']
-------
3668770 ['Санкт-Петербург', 'Фрунзенский район', 'Лиговский просп.', 'д 232']
-------
3657867 ['Санкт-Петербург', 'Приморский район', 'Коломяжский пр.', '15к1']
-------
3682027 ['Санкт-Петербург', 'Красногвардейский район', 'Муринская дор.', '326']
-------


In [ ]:
print(flats['3689147']['address'].split(','))

['Санкт-Петербург', ' Красносельский район', ' Ленинский проспект', ' 55 2']


In [ ]:
all_ids = sorted(list(flats.keys()))

In [ ]:
counter = 0
for id in all_ids:
    if flats[id]['address'] == None:
        print(id, flats[id]['address'])
        print('--------')
        counter += 1
    elif len(flats[id]['address'].split(', ')) == 1:
        print(id, flats[id]['address'])
        print('--------')

3654672 Санкт-Петербург
--------
3689064 None
--------
3696085 Санкт-Петербург
--------


In [ ]:
print(flats['3697566']['address'])

In [ ]:
flats['3680143']

In [ ]:
# dataset formation
counter = 0
new_flats = dict()
chars = ['total_area', 'living_area', 'kitchen_area', 'renovation', 'ceiling_height', 'balcony',  'view', 
         'bathroom', 'hot_water', 'heating', 'stove', 'storeys', 'house_type', 'year_built', 'year_repaired', 
         'elevator', 'entrance', 'parking', 'security']
for id in flats:
    if id == '3689064': # somewhat broken id :(
        continue
    info = flats[id]
    new_info = dict()
    new_info['address'] = info['address']
    new_info['company'] = info['company']
    new_info['agent'] = info['agent']
    for i in range(len(chars)):
        new_info[chars[i]] = info['characteristics'][i]
    if info['close_metro'] == None:
        new_info['close_metro1'] = None
        new_info['close_metro2'] = None
        new_info['close_railway1'] = None
        new_info['close_railway2'] = None
        new_info['close_metro1_dist'] = None
        new_info['close_metro2_dist'] = None
        new_info['close_railway1_dist'] = None
        new_info['close_railway2_dist'] = None
    else:
        new_info['close_metro1'] = info['close_metro'][0]
        new_info['close_metro2'] = info['close_metro'][1]
        new_info['close_railway1'] = info['close_railway'][0]
        new_info['close_railway2'] = info['close_railway'][1]
        new_info['close_metro1_dist'] = info['transport_dist'][0]
        new_info['close_metro2_dist'] = info['transport_dist'][1]
        new_info['close_railway1_dist'] = info['transport_dist'][2]
        new_info['close_railway2_dist'] = info['transport_dist'][3]
    new_info['desc'] = info['desc']
    new_info['how_selled'] = info['how_selled']
    new_info['metro'] = info['metro']
    new_info['name'] = info['name']
    # history features
    # update_history
    new_info['first_seen'] = info['update_history'][0][1]
    n_updates = 0
    last_seen = None
    for obs in info['update_history']:
        if obs[0] == None:
            continue
        yesterday = obs[0].find('вчера')
        today = obs[0].find('сегодня')
        if yesterday != -1 or today != -1:
            n_updates += 1
        if obs[0] != None:
            last_seen = obs[1]
    new_info['n_updates'] = n_updates
    new_info['last_seen'] = last_seen
    # price_history
    new_info['first_price'] = prep_digit(info['price_history'][0][0])
    last_price = None
    for obs in info['price_history']:
        if obs[0] == None:
            continue
        last_price = prep_digit(obs[0])
    new_info['last_price'] = last_price
    # info1_history 
    owner = False
    online_display = 0
    for obs in info['info1_history']:
        if obs[0] == None:
            continue
        for feature in obs[0]:
            if feature == 'собственник':
                owner = True
            elif feature == 'онлайн показы':
                online_display += 1
    new_info['owner'] = owner
    new_info['online_display'] = online_display
    # info2_history
    top = 0
    up = 0
    color = 0
    day = 0
    warn = 0
    for obs in info['info2_history']:
        if obs[0] == None:
            continue
        for feature in obs[0]:
            if feature == 'top':
                top += 1
            elif feature == 'up':
                up += 1
            elif feature == 'color':
                color += 1
            elif feature == 'day':
                day += 1  
            elif feature == 'warn':
                warn += 1
    new_info['top'] = top
    new_info['up'] = up
    new_info['color'] = color
    new_info['day'] = day
    new_info['warn'] = warn
    new_flats[int(id)] = new_info

In [ ]:
df = pd.DataFrame.from_dict(new_flats, orient='index')

In [ ]:
# dataset improvement
# address : разбираемся с городами и деревнями внутри Петербурга и Ленинградской области
df['area'] = df['address'].str.split(pat = ', ', expand=True)[0]
df['district'] = df['address'].str.split(pat = ', ', expand=True)[1]
df['unit'] = df['address'].str.split(pat = ', ', expand=True)[2]
spb_cities = ['Левашово', 'Парголово', 'Колпино', 'Металлострой', 'Петро-Славянка', 'Понтонный', 'Сапёрный', 'Усть-Ижора', 
               'Красное Село', 'Кронштадт', 'Белоостров', 'Зеленогорск', 'Комарово', 'Молодёжное', 'Песочный', 'Репино',
               'Серово', 'Сестрорецк', 'Смолячково', 'Солнечное', 'Ушково', 'Ломоносов', 'Петергоф', 'Стрельна', 'Лисий Нос',
               'Александровская', 'Павловск', 'Пушкин', 'Тярлево', 'Шушары']
suburb_citites = ['Гатчина', 'Мурино', 'Всеволожск', 'Выборг', 'Сосновый Бор', 'Сертолово', 'Тихвин', 'Кириши', 'Кудрово', 'Кингисепп', 
                  'Волхов', 'Тосно', 'Луга', 'Сланцы', 'Кировск', 'Отрадное', 'Никольское', 'Коммунар', 'Пикалёво', 'Лодейное Поле', 
                  'Приозерск', 'Подпорожье', 'Светогорск', 'Бокситогорск', 'Шлиссельбург', 'Сясьстрой', 'Волосово', 'Ивангород', 'Новая Ладога', 
                  'Каменногорск', 'Приморск', 'Любань', 'Высоцк', 'Рощино', 'Янино-1', 'Ульяновка', 'имени Свердлова', 'Сиверский', 'Вырица', 
                  'Кузьмоловский', 'имени Морозова', 'Мга', 'Дубровка', 'Советский', 'Форносово', 'Новоселье', 'Приладожский', 'Токсово', 
                  'Красный Бор', 'Виллози', 'Назия', 'Лебяжье', 'Синявино', 'Кузнечное', 'Фёдоровское', 'Рахья', 'Будогощь', 'Дружная Горка', 
                  'Павлово', 'Ефимовский', 'Лесогорский', 'Рябово', 'Тайцы', 'Большая Ижора', 'Толмачёво', 'Никольский', 'Важины', 'Вознесенье', 
                  'Свирьстрой']
unit = []
for i, row in df.iterrows():
    if row['unit'] == None:
        unit.append(None)
    elif row['area'] == 'Ленинградская область':
        flag = True
        for city in suburb_citites:
            if row['unit'].find(city) != -1:
                unit.append(city)
                flag = False
                break
        if flag:
            unit.append('уездный город N')
    elif row['area'] == 'Санкт-Петербург':
        flag = True
        for city in spb_cities:
            if row['unit'].find(city) != -1:
                unit.append(city)
                flag = False
                break
        if flag:
            unit.append('какая-то улица')
df['unit'] = unit
# agent в исправлении не нуждается
# company
df.loc[df['company'] == 'Частное.', 'company'] = 'частное' # исправляем ошибки 
df.loc[df['company'] == 'собственник', 'company'] = 'частное' # исправляем ошибки 
# areas: удаляем кв. метры
df['total_area'] = df['total_area'].str.split(expand=True)[0]
df['living_area'] = df['living_area'].str.split(expand=True)[0]
df['kitchen_area'] = df['kitchen_area'].str.split(expand=True)[0]
df['total_area'] = df['total_area'].astype('float')
df['living_area'] = df['living_area'].astype('float')
df['kitchen_area'] = df['kitchen_area'].astype('float')
# renovation
df.loc[df['renovation'] == '59.20 кв.м.', 'renovation'] = None # исправляем ошибки 
# ceiling_height
df['ceiling_height'] = df['ceiling_height'].str.split(expand=True)[0]
df['ceiling_height'] = df['ceiling_height'].astype('float')
# balcony, view, bathroom в исправлении не нуждаются. Но их надо будет изменить для вставки в модель
# hot_water, heating, stove в исправлении не нуждаются
# storeys : присутствует не та информация (о названии компании), некоторые этажи написаны не в том формате.
storeys = []
for i, row in df.iterrows():
    if row['storeys'] == None:
        storeys.append(None)
        continue
    splitted = row['storeys'].split(',')
    if len(splitted) == 1:
        found = splitted[0].find('/')
        if found == -1:
            storeys.append(None)
        else:
            storeys.append(splitted[0])
    elif len(splitted) == 2:
        storeys.append(splitted[1])
df['storeys'] = storeys
df['storey'] = df['storeys'].str.split(pat = '/', expand=True)[0]
df['storeys'] = df['storeys'].str.split(pat = '/', expand=True)[1]
# house_type: основная ошибка - запись этажности в эту колоноку
house_type = []
for i, row in df.iterrows():
    if row['house_type'] == None:
        house_type.append(None)
        continue
    found = row['house_type'].find('/')
    if found != -1:
        house_type.append(None)
    else:
        house_type.append(row['house_type'])
df['house_type'] = house_type
# year_built: присутствуют значения из других колонок
df.loc[df['year_built'] == 'сдан', 'year_built'] = '2021' # делаем предположение о том, что дома со значением "сдан" были сданы в 2021
df['year_built'] = pd.to_numeric(df['year_built'], errors='coerce')
# year_repaired: присутствуют значения из других колонок
df['year_repaired'] = pd.to_numeric(df['year_repaired'], errors='coerce')
# elevator: присутствуют значения из других колонок. Надо будет изменить эту колонку для вставки в модель
df.loc[df['elevator'] == '1970', 'elevator'] = None
# entrance, parking: в исправлении не нуждаются. Однако надо будет изменить эти колонки для вставки в модель
# security, close_metro1, close_metro2, close_railway1, close_railway2: в исправлении не нуждаются
# distances: удаляем километры
df['close_metro1_dist'] = df['close_metro1_dist'].str.split(expand=True)[0]
df['close_metro2_dist'] = df['close_metro2_dist'].str.split(expand=True)[0]
df['close_railway1_dist'] = df['close_railway1_dist'].str.split(expand=True)[0]
df['close_railway2_dist'] = df['close_railway2_dist'].str.split(expand=True)[0]
# how_selled: возможно стоит подредактировать перед вставкой в модель
# metro: не нужно исправлять
# name: стоит подредактировать перед вставкой в модель
# first_seen, n_updates, last_seen, first_price, last_price, owner, online_display, top, up, color, day, warn: в исправлении не нуждаются
str_columns = ['address', 'desc']
cat_columns = ['company', 'agent', 'renovation', 'balcony', 'view', 'bathroom', 'hot_water', 'heating', 'stove', 
               'house_type', 'elevator', 'entrance', 'parking', 'security', 'close_metro1', 'close_metro2',
               'close_railway1', 'close_railway2', 'how_selled', 'metro', 'name', 'area', 'district', 'unit']
int_columns = ['storeys', 'year_built', 'year_repaired', 'n_updates', 'first_price', 'last_price', 'owner',
               'online_display', 'top', 'up', 'color', 'day', 'warn', 'storey']
float_columns = ['total_area', 'living_area', 'kitchen_area', 'ceiling_height', 'close_metro1_dist', 	
                 'close_metro2_dist', 'close_railway1_dist', 'close_railway2_dist']
datetime_columns = ['first_seen', 'last_seen']
for col in str_columns:
    df[col] = df[col].astype('str')
for col in cat_columns:
    df[col] = df[col].astype('category')
for col in int_columns:
    df[col] = df[col].astype('float').astype('Int64')
for col in float_columns:
    df[col] = df[col].astype('float')
for col in datetime_columns:
    df[col] = pd.to_datetime(df[col], dayfirst = True)
df = df[['name', 'address', 'area', 'district', 'unit', 'company', 'agent', 'total_area',
        'living_area', 'kitchen_area', 'ceiling_height', 'renovation', 'balcony', 'view',
       'bathroom', 'hot_water', 'heating', 'stove', 'storey', 'storeys',
       'house_type', 'year_built', 'year_repaired', 'elevator',
       'entrance', 'parking', 'security', 'close_metro1', 'close_metro1_dist',
      'close_metro2', 'close_metro2_dist', 'close_railway1', 'close_railway1_dist',
      'close_railway2', 'close_railway2_dist', 'desc', 'how_selled', 'metro', 'first_seen', 'last_seen',
       'n_updates', 'first_price', 'last_price', 'owner',
       'online_display', 'top', 'up', 'color', 'day', 'warn']]

In [ ]:
df[df['last_seen'] <= '2021-05-11']

,name,address,area,district,unit,company,agent,total_area,living_area,kitchen_area,ceiling_height,renovation,balcony,view,bathroom,hot_water,heating,stove,storey,storeys,house_type,year_built,year_repaired,elevator,entrance,parking,security,close_metro1,close_metro1_dist,close_metro2,close_metro2_dist,close_railway1,close_railway1_dist,close_railway2,close_railway2_dist,desc,how_selled,metro,first_seen,last_seen,n_updates,first_price,last_price,owner,online_display,top,up,color,day,warn
3653399,3 комнатная квартира,"Ленинградская область, Выборгский район, Возро...",Ленинградская область,Выборгский район,уездный город N,Центр Вторичного Жилья,NaN,54.5,34.7,6.4,2.48,NaN,Нет,На улицу,Совмещенный,NaN,NaN,NaN,2,2,Кирпичный,1968,<NA>,NaN,NaN,NaN,NaN,Парнас,116.2,Проспект Просвещения,117.5,Возрождение,0.7,21 км (Выборгское напр-е),3.5,При покупке этой квартиры Вы получаете скидку ...,", прямая продажа, ипотека",NaN,2021-04-14,2021-04-14,1,1500000,1500000,0,0,0,0,0,0,0
3654359,1 комнатная квартира,"Ленинградская область, Волховский район, Потан...",Ленинградская область,Волховский район,уездный город N,Петербургская недвижимость,NaN,34.0,16.9,7.4,NaN,NaN,Балкон,Во двор,Раздельный,NaN,NaN,NaN,2,3,Панельный,1987,<NA>,Есть,NaN,NaN,NaN,Девяткино,131.5,Гражданский проспект,133.2,Юги,3.5,Сидорово,5.9,Прямая продажа. Прекрасный вариант как для пос...,", прямая продажа, ипотека",NaN,2021-04-14,2021-05-06,22,750000,750000,0,0,0,0,0,0,0
3670838,Студия,"Ленинградская область, Ломоносовский район, Ан...",Ленинградская область,Ломоносовский район,уездный город N,Русский Фонд Недвижимости,NaN,21.6,14.6,NaN,2.70,Черновая отделка,Лоджия,NaN,Совмещенный,NaN,NaN,NaN,2,4,Кирпично-Монолитный,2020,<NA>,Нет,NaN,NaN,NaN,Проспект Ветеранов,13.7,Ленинский проспект,15.0,Горелово,4.4,Скачки,4.6,В продаже Квартира-студия. ЖК `Шоколад`\nНовый...,", встречная покупка, ипотека",Проспект Ветеранов,2021-04-14,2021-05-05,2,2350000,2350000,0,0,0,0,0,0,0
3555091,1 комнатная квартира,"Ленинградская область, Всеволожский район, Лес...",Ленинградская область,Всеволожский район,уездный город N,Русский Фонд Недвижимости,NaN,27.0,16.0,5.5,NaN,Частичный ремонт,Нет,NaN,Совмещенный,NaN,NaN,NaN,1,3,Кирпичный,1961,<NA>,Нет,NaN,NaN,NaN,Парнас,17.0,Проспект Просвещения,18.3,Дибуны,6.9,Песочная,7.5,предлагается к продаже однокомнатная квартира ...,", прямая продажа, ипотека",Беговая,2021-04-14,2021-04-23,4,2300000,1690000,0,0,0,0,0,0,0
3639293,1 комнатная квартира,"Ленинградская область, Всеволожский район, Лес...",Ленинградская область,Всеволожский район,уездный город N,Русский Фонд Недвижимости,NaN,27.6,17.0,5.5,NaN,Не требуется ремонт,Нет,NaN,Совмещенный,NaN,NaN,NaN,1,3,Кирпичный,1961,<NA>,Нет,NaN,NaN,NaN,Парнас,35.4,Проспект Просвещения,37.1,54 км (Приозерское напр-е),7.9,Лемболово,8.1,выставлена в продажу однокомнатная квартира на...,", прямая продажа, ипотека",Парнас,2021-04-14,2021-04-19,4,2205000,1800000,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3693433,1 комнатная квартира,"Санкт-Петербург, Красносельский район, Краснос...",Санкт-Петербург,Красносельский район,какая-то улица,Адвекс,Губайдулина А.Х.,42.0,16.5,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,17,Кирпичный,<NA>,<NA>,NaN,NaN,NaN,NaN,Проспект Ветеранов,5.3,Автово,5.3,Сосновая Поляна,4.2,Лигово,4.3,"id=1765246 Отличная квартира,с ремонтом,постан...",", прямая продажа, ипотека",Ленинский проспект,2021-05-11,2021-05-11,1,5500000,5500000,0,0,0,0,0,0,0
3693635,1 комнатная квартира,"Санкт-Петербург, Петродворцовый район, Петерго...",Санкт-Петербург,Петродворцовый район,Петергоф,Русский Фонд Недвижимости,NaN,31.7,16.7,8.4,2.60,Не требуется ремонт,Нет,NaN,Совмещенный,NaN,NaN,NaN,4,5,Кирпичный,1962,<NA>,Нет,NaN,NaN,NaN,Новокрестовская,21.9,Приморская,22.1,Старый Петергоф,0.3,Университет,1.7,"Санкт-Петербург, г. Петергоф, ул. Халтурина 9\...",", встречная поку

In [ ]:
df.to_csv('flats.csv')
flts = pd.read_csv('flats.csv', index_col = 0)

In [ ]:
flts.head()

,name,address,area,district,unit,company,agent,total_area,living_area,kitchen_area,ceiling_height,renovation,balcony,view,bathroom,hot_water,heating,stove,storey,storeys,house_type,year_built,year_repaired,elevator,entrance,parking,security,close_metro1,close_metro1_dist,close_metro2,close_metro2_dist,close_railway1,close_railway1_dist,close_railway2,close_railway2_dist,desc,how_selled,metro,first_seen,last_seen,n_updates,first_price,last_price,owner,online_display,top,up,color,day,warn
3653407,1 комнатная квартира,"Ленинградская область, Выборгский район, Вещев...",Ленинградская область,Выборгский район,уездный город N,Центр Вторичного Жилья,NaN,31.3,11.6,8.50,2.55,Требуется ремонт,Нет,На улицу,Раздельный,NaN,NaN,NaN,1.0,5.0,Панельный,1985.0,NaN,NaN,NaN,NaN,NaN,Парнас,92.2,Проспект Просвещения,93.5,Гаврилово,13.7,Лейпясуо,14.9,При покупке этой квартиры Вы получаете скидку ...,", прямая продажа, ипотека",NaN,2021-04-14,2021-05-18,34,700000,700000,0,0,0,0,0,0,0
3653398,2 комнатная квартира,"Ленинградская область, Ломоносовский район, Ло...",Ленинградская область,Ломоносовский район,уездный город N,Центр Вторичного Жилья,NaN,42.6,27.3,5.20,2.50,NaN,Нет,На улицу,Раздельный,NaN,NaN,NaN,1.0,5.0,Панельный,1968.0,NaN,NaN,NaN,NaN,NaN,Проспект Ветеранов,48.8,Ленинский проспект,49.8,Калище,24.1,Карьер 75 км (Балтийское и Варшавское напр-е),24.7,При покупке этой квартиры Вы получаете скидку ...,", прямая продажа, ипотека",Автово,2021-04-14,2021-05-18,34,1925000,1925000,0,0,0,0,0,0,0
3653399,3 комнатная квартира,"Ленинградская область, Выборгский район, Возро...",Ленинградская область,Выборгский район,уездный город N,Центр Вторичного Жилья,NaN,54.5,34.7,6.40,2.48,NaN,Нет,На улицу,Совмещенный,NaN,NaN,NaN,2.0,2.0,Кирпичный,1968.0,NaN,NaN,NaN,NaN,NaN,Парнас,116.2,Проспект Просвещения,117.5,Возрождение,0.7,21 км (Выборгское напр-е),3.5,При покупке этой квартиры Вы получаете скидку ...,", прямая продажа, ипотека",NaN,2021-04-14,2021-04-14,1,1500000,1500000,0,0,0,0,0,0,0
3674279,1 комнатная квартира,"Ленинградская область, Выборгский район, Выбор...",Ленинградская область,Выборгский район,Выборг,Центр Вторичного Жилья,NaN,32.0,17.9,7.90,2.50,NaN,Нет,На улицу,Совмещенный,NaN,NaN,NaN,2.0,2.0,Деревянный,1940.0,NaN,NaN,NaN,NaN,NaN,Парнас,114.5,Беговая,115.2,Выборг,1.5,2 км (Выборгское напр-е),2.8,"Рады, что вы открыли данное объявление, потому...",", прямая продажа, ипотека",NaN,2021-04-14,2021-05-18,34,1700000,1700000,0,0,0,0,0,0,0
3653389,1 комнатная квартира,"Ленинградская область, Выборгский район, Выбор...",Ленинградская область,Выборгский район,Выборг,Центр Вторичного Жилья,NaN,38.0,14.0,17.51,2.67,NaN,Нет,На улицу,Раздельный,NaN,NaN,NaN,1.0,2.0,Деревянный,1940.0,NaN,NaN,NaN,NaN,NaN,Парнас,114.6,Беговая,115.4,Выборг,1.7,2 км (Выборгское напр-е),2.9,При покупке этой квартиры Вы получаете скидку ...,", прямая продажа, ипотека",NaN,2021-04-14,2021-05-18,34,2500000,2500000,0,0,0,0,0,0,0


In [ ]:
#wrong = []
#for id in all_ids:
#    if len(flats[id]['address_history']) > 1:
#        if (flats[id]['address_history'][-1][0] != flats[id]['address_history'][-2][0]) and (flats[id]['address_history'][-1][0] != None) and (flats[id]['address_history'][-2][0] != None):
#            wrong.append(id)